In [1]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.random_sampling import RandomSampling
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework2 import Framework2
from copy import deepcopy
import torch.nn as nn
import time

In [2]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [3]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [4]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=100 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=10 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)
model_name = sorted(name for name in models.__dict__ 
                    if callable(models.__dict__[name]))
model_name
model=Model('resnet50').get_model()
model
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

6000 90 600


array([False, False, False, ..., False,  True, False])

In [5]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [6]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [7]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [8]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [9]:
# al_strat=RandomSampling(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
# ssl_strat=fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [10]:
framework_2= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args,RandomSampling,fixmatch)

In [11]:
framework_2.train(2e-3,10)

Let's use 1 GPUs!
[Batch=000] [Loss=2.59]
[Batch=010] [Loss=11.40]
[Batch=020] [Loss=2.31]
[Batch=030] [Loss=2.13]
[Batch=040] [Loss=12.44]

==>>[2023-08-18 07:46:48] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=2.11]
[Batch=010] [Loss=2.09]
[Batch=020] [Loss=1.96]
[Batch=030] [Loss=20.55]
[Batch=040] [Loss=12.35]

==>>[2023-08-18 07:46:52] [Epoch=001/010] [framwork1 Need: 00:00:40] [LR=0.1000] [Best : Test Accuracy=0.13, Error=0.87]
[Batch=000] [Loss=4.67]
[Batch=010] [Loss=15.57]
[Batch=020] [Loss=3.16]
[Batch=030] [Loss=6.85]
[Batch=040] [Loss=24.76]

==>>[2023-08-18 07:46:55] [Epoch=002/010] [framwork1 Need: 00:00:30] [LR=0.1000] [Best : Test Accuracy=0.13, Error=0.87]
[Batch=000] [Loss=1.85]
[Batch=010] [Loss=1.87]
[Batch=020] [Loss=1.84]
[Batch=030] [Loss=12.39]
[Batch=040] [Loss=9.47]

==>>[2023-08-18 07:46:58] [Epoch=003/010] [framwork1 Need: 00:00:24] [LR=0.1000] [Best : Test Accuracy=0.25, Error=0.75]
[Batch=

0.8105

In [12]:
test_acc=framework_2.predict(X_te,Y_te)

In [13]:
test_acc

0.8105

In [14]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [15]:
len(acc)

91

In [16]:
for rd in range(1, NUM_ROUND+1):

    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = framework_2.stratAl.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = framework_2.stratSSL.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc


Round 1/90
Let's use 1 GPUs!
[Batch=000] [Loss=1.06]
[Batch=010] [Loss=0.77]
[Batch=020] [Loss=0.80]

==>>[2023-08-18 07:47:41] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=0.75]
[Batch=010] [Loss=0.93]
[Batch=020] [Loss=0.65]

==>>[2023-08-18 07:48:05] [Epoch=001/010] [framwork1 Need: 00:03:33] [LR=0.1000] [Best : Test Accuracy=0.85, Error=0.15]
[Batch=000] [Loss=0.80]
[Batch=010] [Loss=0.81]
[Batch=020] [Loss=0.75]

==>>[2023-08-18 07:48:29] [Epoch=002/010] [framwork1 Need: 00:03:10] [LR=0.1000] [Best : Test Accuracy=0.85, Error=0.15]
[Batch=000] [Loss=0.70]
[Batch=010] [Loss=0.69]
[Batch=020] [Loss=0.77]

==>>[2023-08-18 07:48:54] [Epoch=003/010] [framwork1 Need: 00:02:47] [LR=0.1000] [Best : Test Accuracy=0.87, Error=0.13]
[Batch=000] [Loss=0.76]
[Batch=010] [Loss=0.76]
[Batch=020] [Loss=0.76]

==>>[2023-08-18 07:49:18] [Epoch=004/010] [framwork1 Need: 00:02:24] [LR=0.1000] [Best : Test Accuracy=0.89, Error=0.11]
[

AttributeError: 'RandomSampling' object has no attribute 'query'

In [ ]:
acc

In [ ]:
import gc

In [ ]:
gc.collect()